<h1>1. Téléchargement du dataset<h1>
<h3>1.1. Acces aux fichiers compréssés <h3>


In [ ]:
!wget http://images.cocodataset.org/zips/train2014.zip
!wget http://images.cocodataset.org/zips/val2014.zip

<h3>1.2. Décompression des fichiers <h3>

In [ ]:
!unzip /content/train2014.zip >train.txt
!unzip /content/val2014.zip >val.txt
""" les fichiers .txt aident à vérifier l'état du processus  """
!rm -v /content/train2014.zip
!rm -v /content/val2014.zip 

<h3>1.3. Chargement des informations du dataset <h3>

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1tRaTpuhuIeDFGZZNjOjfIch232Cr2uMS' -O dataset.pickle
import pickle
dataset_path='/content/dataset.pickle'
file = open(dataset_path, 'rb')
[dict_id,dict_data]=pickle.load(file)
file.close()
print('train size:',len(dict_id['train']))
print('validation size:',len(dict_id['val']))


--2020-09-15 20:21:06--  https://docs.google.com/uc?export=download&id=1tRaTpuhuIeDFGZZNjOjfIch232Cr2uMS
Resolving docs.google.com (docs.google.com)... 172.253.119.138, 172.253.119.102, 172.253.119.113, ...
Connecting to docs.google.com (docs.google.com)|172.253.119.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-6c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tvkj7kquk8cuq4hfvr0ho5cnk5fe8g1r/1600201200000/15956116802716578091/*/1tRaTpuhuIeDFGZZNjOjfIch232Cr2uMS?e=download [following]
--2020-09-15 20:21:08--  https://doc-04-6c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tvkj7kquk8cuq4hfvr0ho5cnk5fe8g1r/1600201200000/15956116802716578091/*/1tRaTpuhuIeDFGZZNjOjfIch232Cr2uMS?e=download
Resolving doc-04-6c-docs.googleusercontent.com (doc-04-6c-docs.googleusercontent.com)... 108.177.111.132, 2607:f8b0:4001:c07::84
Connecting to doc-04-6c-docs.googleusercontent.com (doc-04

<h1>2. Partie Encodage <h1>


<h3>2.1 téléchargement des outils TF API <h3>

In [ ]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd models/research
pip install .

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1


# Patch the location of gfile
tf.gfile = tf.io.gfile

<h3>2.2 Chargement du modèle ssd_mobilenet_v1_coco_2017_11_17 <h3>

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)
print(detection_model.signatures['serving_default'].inputs)

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:

def get_features(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path).convert('RGB'))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Return the results of a detection.
  return np.array([ output_dict['detection_boxes'][0:30].reshape(-1),
      output_dict['detection_classes'][0:30],
      output_dict['detection_scores'][0:30]])

<h3>2.3 Encodage des images ( Extraction des caractéristiques ) <h3>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

features=[]
ids=[]
print('extracting train features')
i=1
for id,instance in dict_data['train'].items():
  #lecture d'image à partir du dossier /content/train2014
  file_name=instance['url'].split('/')[-1]
  image_path='/content/train2014/'+file_name
  ids.append(id)
  features.append(get_features(detection_model, image_path))
  if i%500==0 :
    print(i//500)
  i+=1
pickle_file = open('/content/drive/My Drive/Projet/train.pickle', 'wb')
pickle.dump((ids,features),pickle_file)
pickle_file.close()
print('end')
#22:11:20

In [ ]:
print('extracting val features')
features=[]
ids=[]
i=1
for id,instance in dict_data['val'].items():
  #lecture d'image à partir du dossier /content/val2014
  file_name=instance['url'].split('/')[-1]
  image_path='/content/val2014/'+file_name
  if i%500==0 :
    print(i//500)
  i+=1
  ids.append(id)
  features.append(get_features(detection_model, image_path))
pickle_file = open('/content/drive/My Drive/Projet/val.pickle', 'wb')
pickle.dump((ids,features),pickle_file)
pickle_file.close()
print('end')
#19:32:00

<h1>3. Partie décodage<h1>

3.1 Extraction des caractères :

In [ ]:
!pip install nltk
import nltk
from nltk import FreqDist
from nltk.tokenize import word_tokenize
import random
import pickle
nltk.download('punkt')
# max size 250
# max min size 70
char_list =dict()
char_list['begin']=0
char_list['None']=1
i=2

freq_char=nltk.FreqDist([char for id in dict_data['train'] for caption in dict_data['train'][id]['captions'] for char in caption.lower()  ])
for char in sorted(set([char for id in dict_data['train'] for caption in dict_data['train'][id]['captions'] for char in caption.lower()  ])):
  if(freq_char[char] >1000 ):
    char_list[char]=i
    i+=1
print(len(char_list))
print(char_list)


In [ ]:
import pickle
import numpy as np
import random

selected_captions=dict()
for id in dict_data['train'] :
  index_list =[ i for i in range(0,len(dict_data['train'][id]['captions'])) if len(dict_data['train'][id]['captions'][i])<100 ]
  random.shuffle(index_list)
  selected_captions[id]=dict_data['train'][id]['captions'][index_list[0]]
pickle_file = open('/content/drive/My Drive/Projet/selectedCaptions.pickle', 'wb')
pickle.dump((selected_captions,char_list),pickle_file)
pickle_file.close()



<h3>3.2 Code d'un générateur de données <h3>

In [ ]:
import pickle
def toarray( nparray_list):
  result=[]
  for array in nparray_list :
    result.extend(array)
  return result
train_features                      = pickle.load(open('/content/drive/My Drive/Projet/train.pickle','rb'))
(selected_captions,char_list)       = pickle.load(open('/content/drive/My Drive/Projet/selectedCaptions.pickle', 'rb'))

def gen_batch( batch_size,char_list,train_features,selected_captions):
  max_sentence_len=100
  inputs=[]
  targets=[]
  for row in range(0,len(train_features[0])):
    if row%batch_size == 0 :
      inputs=[]
      targets=[]
    img_features= train_features[1][row]
    img_features= toarray(img_features)
    id= train_features[0][row]
    caption=selected_captions[id]
    input=[]
    target=[]
    char_index=char_list['begin'] 
    for char in caption:
      if char in char_list:
        codif_data=[]
        codif_data.extend( img_features )
        codif_data.append(char_index)
        input.append(codif_data)
        char_index=char_list[char] 
        target.append(char_index)

    while len(input)<max_sentence_len:
      codif_data=[]
      codif_data.extend( img_features )
      codif_data.append(char_index)
      input.append(codif_data)
      char_index=char_list['None'] 
      target.append(char_index)

    inputs.append(input)
    targets.append(target)

    if (row+1)%batch_size == 0 :
      yield (np.array(inputs),np.array(targets))
  if (len(train_features[0]))%batch_size != 0 and False:
    yield (np.array(inputs),np.array(targets))


"""
  dict_data                   : dictionnaire contient les données sur le dataset ,
  batch_size                  : taille du batch in 64,128,256,512 ,
  words_list                  : liste des mots du vocab utilisé ,
"""



<h3>3.3 création du modèle <h3>

In [ ]:

from keras.models import Sequential
from keras.layers import Dropout,LSTM,Dense
import tensorflow as tf
import pickle
import numpy as np


vocab_size= len(char_list)
def toarray( nparray_list):
  result=[]
  for array in nparray_list :
    result.extend(array)
  return result

"""
  dict_data                   : dictionnaire contient les données sur le dataset ,
  batch_size                  : taille du batch in 64,128,256,512 ,
  words_list                  : liste des mots du vocab utilisé ,
"""

model = Sequential( name='RNN' )
model.add(  tf.keras.Input(shape=(100,181), batch_size=64) )
model.add(  LSTM(128, return_sequences=True, stateful=True) )
model.add(  Dropout(.2)  )
model.add(  LSTM(128, return_sequences=True, stateful=True) )
model.add(  Dropout(.2)  )
model.add(  Dense(vocab_size, activation="softmax") )
model.summary()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(lr=0.001)
# Loss
train_loss = tf.keras.metrics.Mean(name='train_loss')
# Accuracy
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
model.compile( optimizer=optimizer, loss=loss_object , metrics=['accuracy'] )

model.reset_states()
@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        # Make a prediction on all the batch
        predictions = model(inputs)
        # Get the error/loss on these predictions
        loss = loss_object(targets, predictions)
    # Compute the gradient which respect to the loss
    gradients = tape.gradient(loss, model.trainable_variables)
    # Change the weights of the model
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # The metrics are accumulate over time. You don't need to average it yourself.
    train_loss(loss)
    train_accuracy(targets, predictions)

@tf.function
def predict(inputs):
    # Make a prediction on all the batch
    predictions = model(inputs)
    return predictions

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (64, 100, 128)            158720    
_________________________________________________________________
dropout (Dropout)            (64, 100, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (64, 100, 128)            131584    
_________________________________________________________________
dropout_1 (Dropout)          (64, 100, 128)            0         
_________________________________________________________________
dense (Dense)                (64, 100, 35)             4515      
Total params: 294,819
Trainable params: 294,819
Non-trainable params: 0
_________________________________________________________________


<h3>3.4 Entrainement du modèle <h3>

In [ ]:
batch_size=64
with tf.device("gpu:0"):
  for epoch in range(1,301):
      
      for batch_inputs, batch_targets in gen_batch(batch_size,char_list,train_features,selected_captions):
          train_step(batch_inputs, batch_targets)
      template = '\r Epoch {}, Train Loss: {}, Train Accuracy: {}'
      print(template.format(epoch, train_loss.result(), train_accuracy.result()*100), end="")
      model.reset_states()
      log_file=open('/content/drive/My Drive/Projet/states.log','a+')
      log_file.write(""+str(train_loss.result())+"\t,"+str(train_accuracy.result()*100)+"\n")
      log_file.close()
      if epoch%10==0:
        model.save('/content/drive/My Drive/Projet/decode%d.h5'%(epoch))
        print('  model saved')

#13:52

<h1>4. Phase des tests <h1>

<h3>4.1 Selon la précision <h3>

In [ ]:
import numpy as np

batch_size=64
for batch_inputs, batch_targets in gen_batch(batch_size,char_list,train_features,selected_captions):
  var =np.argmax(model(batch_inputs) ,axis=2)
  for line in var :
    for char in line :
      if char !=1:
        print(liste[char],end='')
    print()


In [ ]:
import tensorflow as tf

def test_model( model_path):
  model=tf.keras.models.load_model(model_path)
  optimizer = tf.keras.optimizers.Adam(lr=0.001)
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
  # Loss
  train_loss = tf.keras.metrics.Mean(name='train_loss')
  # Accuracy
  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
  model.compile( optimizer=optimizer, loss=loss_object , metrics=['accuracy'] )
  batch_size=64
  for inputs,targets in  gen_batch(batch_size,char_list,train_features,selected_captions):
    predictions=model(inputs)
    train_accuracy(targets, predictions)
    loss = loss_object(targets, predictions)
    train_loss(loss)
  template = '\r model {}, Train Loss: {}, Train Accuracy: {}'
  print(template.format(model_path, train_loss.result(), train_accuracy.result()*100), end="\n")

for num in range(10,290,10): 
  test_model('/content/drive/My Drive/Projet/decode%d.h5'%(num))



#14.22.00

In [ ]:
import numpy as np 
import pickle
import random
def to_array( nparray_list):
  result=[]
  for array in nparray_list :
    result.extend(array)
  return result
val_features                      = pickle.load(open('/content/drive/My Drive/Projet/val.pickle','rb'))
(selected_captions,char_list)       = pickle.load(open('/content/drive/My Drive/Projet/selectedCaptions.pickle', 'rb'))

def gen_val( batch_size,char_list,val_features,dict_data=dict_data):
  max_sentence_len=100
  inputs=[]
  targets=[]
  for row in range(0,len(val_features[0])):
    if row%batch_size == 0 :
      inputs=[]
      targets=[]
    img_features= val_features[1][row]
    img_features= to_array(img_features)
    id= val_features[0][row]
    captions=dict_data['val'][id]['captions']
    captions = [caption for caption in captions if len(caption)<max_sentence_len ]
    caption = random.choice(captions)
    input=[]
    target=[]
    char_index=char_list['begin'] 
    for char in caption.lower():
      if char in char_list:
        codif_data=[]
        codif_data.extend( img_features )
        codif_data.append(char_index)
        input.append(codif_data)
        char_index=char_list[char] 
        target.append(char_index)

    while len(input)<max_sentence_len:
      codif_data=[]
      codif_data.extend( img_features )
      codif_data.append(char_index)
      input.append(codif_data)
      char_index=char_list['None'] 
      target.append(char_index)

    inputs.append(input)
    targets.append(target)

    if (row+1)%batch_size == 0 :
      yield (np.array(inputs),np.array(targets))
  if (len(val_features[0]))%batch_size != 0 and False:
    yield (np.array(inputs),np.array(targets))


"""
  dict_data                   : dictionnaire contient les données sur le dataset ,
  batch_size                  : taille du batch in 64,128,256,512 ,
  words_list                  : liste des mots du vocab utilisé ,
"""
batch_size=64
i=0
for inputs,targets in gen_val( batch_size,char_list,val_features):
  print(i)
  i+=1
  break 
#16:23:50

0


<h3>4.2 Selon la F-mesure moyenne <h3>

In [ ]:
from sklearn.metrics import f1_score
import tensorflow as tf
import numpy as np
def test_f_mesure(model_path):
  model=tf.keras.models.load_model(model_path)
  all_targets=[]
  all_outputs=[]
  for inputs,targets in gen_val( batch_size,char_list,val_features):
      all_targets.extend(targets.reshape(-1))
      outputs=model(inputs)
      outputs=np.argmax(outputs, axis=-1).reshape(-1)
      all_outputs.extend(outputs)
  template = '\r model {}, weigheted: {}, macro: {}'
  labels=[i for i in range(0,len(char_list)) if not i==1]
  print(template.format(model_path,f1_score(all_targets, all_outputs,labels=labels, average='weighted'),f1_score(all_targets, all_outputs,labels=labels, average='macro')), end="\n")
#12:20:50

In [ ]:
for num in range(10,290,10): 
  test_f_mesure('/content/drive/My Drive/Projet/decode%d.h5'%(num))
#15:43

<h3>4.1 Selon la mesure Bleu-N <h3>

In [ ]:
import nltk
import tensorflow as tf 
import numpy as np 
import pickle

def to_array( nparray_list):
  result=[]
  for array in nparray_list :
    result.extend(array)
  return result
model_path='/content/drive/My Drive/Projet/decode280.h5'
model=tf.keras.models.load_model(model_path)
val_features                      = pickle.load(open('/content/drive/My Drive/Projet/val.pickle','rb'))
(_,char_list)       = pickle.load(open('/content/drive/My Drive/Projet/selectedCaptions.pickle', 'rb'))

def predict_sent( model, features,char_list):
  sent=""
  last_output=char_list['begin'] 
  keys_list=[key for key in char_list]
  i=0
  input=[]
  features=to_array(features)
  input.extend(features)
  input.append(last_output)
  model.reset_states()
  data=np.zeros((64,100,181))
  data[0,i]=np.array(input)
  output=model(data)
  last_output=np.argmax(output[0,i])
  while last_output != char_list['None'] and i<99:
    sent+=keys_list[last_output]
    input=[]
    input.extend(features)
    input.append(last_output)
    i+=1
    data[0,i]=np.array(input)
    output=model(data)
    last_output=np.argmax(output[0,i])
  return sent
  
def predict_batch( model, features_batch ,char_list):
  model.reset_states()
  sent=[]
  last_output=[]
  for i in range(0,len( features_batch)):
    features_batch[i]=to_array( features_batch[i])
  for i in range(0,64):
    last_output.append(char_list['begin'] )
  keys_list=[key for key in char_list]
  j=0
  data=np.zeros((64,100,181))
  input=[]
  output=[]
  for i in range(0,len( features_batch)):
    input.append([])
    input[i].extend(features_batch[i])
    input[i].append(last_output[i])
  while j<100:
    model.reset_states()
    for i in range(0,len( features_batch)):
      data[i,j]=np.array(input[i])
    output=model(data)
    last_output=[]
    for i in range(0,len( features_batch)):
      last_output.append(np.argmax(output[i,j]))
    input=[]
    for i in range(0,len( features_batch)):
      input.append([])
      input[i].extend(features_batch[i])
      input[i].append(last_output[i])
    j+=1
  result=[]
  output=np.argmax(output,axis=2)
  for i in range(output.shape[0]):
    sent=""
    for j in range(output.shape[1]):
      c=keys_list[output[i,j]]
      if c=='None' :
        break
      sent+=c
    result.append(sent)
  return result
  

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
nltk.download('punkt')
def predict_all_captions(model,val_features,char_list):
  all_captions=[]
  i=0
  for i in range(64,len(val_features),64):
    all_captions.extend(predict_batch( model, val_features[i-64:i] ,char_list))
    print(i)
  i=len(val_features)-len(val_features)%64
  while i<len(val_features):
    sent=predict_sent( model, val_features[i],char_list)
    all_captions.append(sent)
    print(i)
    i+=1
  print(len(val_features))
  return all_captions
all_captions=predict_all_captions(model, val_features[1],char_list)
pickle_file = open('/content/drive/My Drive/Projet/all_captions.pickle', 'wb')
pickle.dump(all_captions,pickle_file)
pickle_file.close()

In [ ]:
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
nltk.download('punkt')
predicted_captions  = pickle.load(open('/content/drive/My Drive/Projet/all_captions.pickle','rb'))

def bleu_1(predicted,references):
  res=[]
  for reference in references :
    freq_ref=FreqDist(word_tokenize( reference ))
    freq_pred=FreqDist(word_tokenize( predicted ))
    count_ref=sum(freq_ref.values())
    count_pred=sum([ min(freq_pred[key],freq_ref[key]) for key in freq_ref.keys() ])
    res.append( count_pred/ count_ref)
  return max(res)

def bleu_n(predicted,references,n):
  res=[]
  for reference in references :
    liste_ref  =[ word for word  in  nltk.ngrams(word_tokenize(reference),n)]
    liste_pred =[ word for word  in  nltk.ngrams(word_tokenize(predicted),n)]
    freq_ref=FreqDist( liste_ref )
    freq_pred=FreqDist( liste_pred )
    count_ref=sum(freq_ref.values())
    count_pred=sum([ min(freq_pred[key],freq_ref[key]) for key in freq_ref.keys() ])
    res.append( count_pred/ count_ref)
  return max(res)

def calc_metriques( predicted_captions, val_features ,dict_data ):
  count=len(val_features[1])
  b1=0
  b2=0
  b3=0
  b4=0
  for i in range(0,count):
    id=val_features[0][i]
    predicted   = predicted_captions[i]
    references  =  dict_data['val'][id]['captions'] 
    b1+=bleu_1(predicted,references)
    b2+=bleu_n(predicted,references,2)
    b3+=bleu_n(predicted,references,3)
    b4+=bleu_n(predicted,references,4)
  return  (b1/count,b2/count,b3/count,b4/count)

print(calc_metriques( predicted_captions, val_features ,dict_data ))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: generator 'ngrams' raised StopIteration


(0.3602033744470266, 0.1214666905397255, 0.03479153426583626, 0.010338026451352279)


<h3>5. Informations sur le Hardware <h3>

In [ ]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=18.04
DISTRIB_CODENAME=bionic
DISTRIB_DESCRIPTION="Ubuntu 18.04.5 LTS"
NAME="Ubuntu"
VERSION="18.04.5 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.5 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic
